# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [1]:
#include <stdio.h>

int main() {
  int num_parcelas = 5;
  float primeira_parcela = 200.0;
  float juros = 1;
  float parcela_atual;

  parcela_atual = primeira_parcela;
  for(int i = 0; i < num_parcelas; i++) {
    printf("%.2f ", parcela_atual);
    parcela_atual = parcela_atual * (1 + juros / 100);
  }
  printf("\n");

  return 0;
}

200.00 202.00 204.02 206.06 208.12 


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include <stdio.h>

float calcula_proxima_parcela(float parcela_atual,
                              float juros,
                              int *parcela_n,
                              int parcela_total) {
  if(*parcela_n == 0) {
    *parcela_n += 1;
    return parcela_atual;
  } else if(*parcela_n < parcela_total) {
    *parcela_n += 1;
    return parcela_atual * (1 + juros / 100);
  } else {
    return 0;
  }
}

int main() {
  float primeira_parcela = 200.0;
  float parcela_atual = 0;
  float juros = 1;
  int parcela_total = 5;
  int parcela_n = 0;

  parcela_atual = primeira_parcela;
  for(int i = 0; i < parcela_total + 1; i++) {
    parcela_atual = calcula_proxima_parcela(parcela_atual,
                                            juros,
                                            &parcela_n,
                                            parcela_total);
    printf("%.2f ", parcela_atual);
  }
  printf("\n");

  return 0;
}

200.00 202.00 204.02 206.06 208.12 0.00 


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [3]:
#include <stdio.h>

float primeira_parcela;
float juros;
float parcela_atual;
int parcela_total;
int parcela_n;

void cria_emprestimo(float nova_primeira_parcela,
                     float novos_juros,
                     int nova_parcela_total) {

  primeira_parcela = nova_primeira_parcela;
  parcela_atual = primeira_parcela;
  juros = novos_juros;
  parcela_total = nova_parcela_total;
  parcela_n = 0;
}

float calcula_proxima_parcela(float parcela_atual,
                              float juros,
                              int *parcela_n,
                              int parcela_total) {
  float parcela = 0;

  if(*parcela_n == 0) {
    parcela = parcela_atual;
  } else if(*parcela_n < parcela_total) {
    parcela = parcela_atual * (1 + juros / 100);
  } else {
    parcela = 0;
  }
  *parcela_n += 1;
  return parcela;
}

int main() {
  cria_emprestimo(200, 1, 5);

  for(int i = 0; i < parcela_total + 1; i++) {
    parcela_atual = calcula_proxima_parcela(parcela_atual,
                                            juros,
                                            &parcela_n,
                                            parcela_total);
    printf("O valor da parcela %d eh %3.2f\n", parcela_n, parcela_atual);
  }

  return 0;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12
O valor da parcela 6 eh 0.00


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [4]:
#include <stdio.h>
#include <stdlib.h>

typedef struct Emprestimo {
  float primeira_parcela;
  float parcela_atual;
  float juros;
  int parcela_total;
  int parcela_n;
} Emprestimo;

Emprestimo *cria_emprestimo(float primeira_parcela,
                            float juros,
                            int parcela_total) {
  Emprestimo *emprestimo = calloc(1, sizeof(Emprestimo));

  emprestimo->primeira_parcela = primeira_parcela;
  emprestimo->parcela_atual = primeira_parcela;
  emprestimo->juros = juros;
  emprestimo->parcela_total = parcela_total;
  emprestimo->parcela_n = 0;

  return emprestimo;
}

float calcula_proxima_parcela(Emprestimo *emprestimo) {
  float parcela = 0;

  if(emprestimo->parcela_n == 0) {
    parcela = emprestimo->parcela_atual;
  } else if(emprestimo->parcela_n < emprestimo->parcela_total) {
    parcela = emprestimo->parcela_atual * (1 + emprestimo->juros / 100);
  } else {
    parcela = 0;
  }
  emprestimo->parcela_n += 1;
  return parcela;
}

int main() {
  Emprestimo *emprestimo = NULL;
  emprestimo = cria_emprestimo(200, 1, 5);

  for(int i = 0; i < emprestimo->parcela_total + 1; i++) {
    emprestimo->parcela_atual = calcula_proxima_parcela(emprestimo);
    printf("O valor da parcela %d eh %3.2f\n", emprestimo->parcela_n, emprestimo->parcela_atual);
  }

  free(emprestimo);

  return 0;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12
O valor da parcela 6 eh 0.00


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [5]:
#include <stdio.h>
#include <stdlib.h>

int id = 0;

typedef struct Emprestimo {
  int id;
  float primeira_parcela;
  float parcela_atual;
  float juros;
  int parcela_total;
  int parcela_n;
} Emprestimo;

Emprestimo *cria_emprestimo(float primeira_parcela,
                            float juros,
                            int parcela_total) {
  Emprestimo *emprestimo = calloc(1, sizeof(Emprestimo));

  emprestimo->id = ++id;
  emprestimo->primeira_parcela = primeira_parcela;
  emprestimo->parcela_atual = primeira_parcela;
  emprestimo->juros = juros;
  emprestimo->parcela_total = parcela_total;
  emprestimo->parcela_n = 0;

  return emprestimo;
}

float calcula_proxima_parcela(Emprestimo *emprestimo) {
  float parcela = 0;

  if(emprestimo->parcela_n == 0) {
    parcela = emprestimo->parcela_atual;
  } else if(emprestimo->parcela_n < emprestimo->parcela_total) {
    parcela = emprestimo->parcela_atual * (1 + emprestimo->juros / 100);
  } else {
    parcela = 0;
  }
  emprestimo->parcela_n += 1;
  return parcela;
}

int main() {
  Emprestimo **emprestimos = calloc(1, sizeof(Emprestimo *));
  Emprestimo *emprestimo1 = cria_emprestimo(200, 1, 5);
  Emprestimo *emprestimo2 = cria_emprestimo(500, 2, 7);
  int total_emprestimos = 2;
  int tem_parcelas = 0;

  *emprestimos = calloc(1, 2 * sizeof(Emprestimo));
  emprestimos[0] = emprestimo1;
  emprestimos[1] = emprestimo2;

  while(tem_parcelas < total_emprestimos) {
    for(int i = 0; i < total_emprestimos; i++) {
      emprestimos[i]->parcela_atual = calcula_proxima_parcela(emprestimos[i]);
      if(emprestimos[i]->parcela_atual == 0) {
        tem_parcelas++;
      }
      if(emprestimos[i]->parcela_atual != 0) {
        printf("Para o emprestimo %d, o valor da parcela %d eh %3.2f\n",
               emprestimos[i]->id,
               emprestimos[i]->parcela_n,
               emprestimos[i]->parcela_atual);
      }
    }
  }

  for(int i = 0; i < 2; i++) {
    free(emprestimos[i]);
  }
  free(emprestimos);

  return 0;
}

Para o emprestimo 1, o valor da parcela 1 eh 200.00
Para o emprestimo 2, o valor da parcela 1 eh 500.00
Para o emprestimo 1, o valor da parcela 2 eh 202.00
Para o emprestimo 2, o valor da parcela 2 eh 510.00
Para o emprestimo 1, o valor da parcela 3 eh 204.02
Para o emprestimo 2, o valor da parcela 3 eh 520.20
Para o emprestimo 1, o valor da parcela 4 eh 206.06
Para o emprestimo 2, o valor da parcela 4 eh 530.60
Para o emprestimo 1, o valor da parcela 5 eh 208.12
Para o emprestimo 2, o valor da parcela 5 eh 541.22
Para o emprestimo 2, o valor da parcela 6 eh 552.04
Para o emprestimo 2, o valor da parcela 7 eh 563.08
